In [4]:
from IPython.display import HTML
HTML('''
<script>
code_show=false;
function code_toggle(){
    if(code_show){$('.prompt, .input, .output_stderr, .output_error').hide();}
    else{$('.input, .prompt, .output_stderr, .output_error').show();}
    code_show=!code_show;
}
function initialize(){
    var output=$('.output_subarea.output_text.output_stream.output_stdout');
    $.merge(output,$('.output_subarea.output_text.output_result'));
    for(var i=0;i<output.length;i++)for(var j=0;j<output[i].children.length;j++)
        output[i].children[j].style.fontFamily='Palatino';
    code_toggle();
}
$(document).ready(initialize);
</script>
Click <a href="javascript:code_toggle()">here</a> to show/hide codes in this notebook.
''')

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact

# Inverse tangent function
$$
    f(x) = arctan(x) \\
    \frac{df}{dx} = \frac{1}{1 + x^2} \\
$$
Assuming a normal distribution on the input $X$~$\mathbf{N}(\mu,\sigma)$ the uncertainty propagation to $Y$ follows:
$$
    \mu_y = arctan(\mu_x) \\
    \sigma_y = \left. \frac{df}{dx}\right|_{\mu_x} \sigma_x
$$

In [6]:
@interact(mean=(-10.0,10.0), variance=(1e-9,6.0))
def show_linear(mean=0.0, variance=1.0):

    x = np.linspace(-10,10, 100)

    pdf1 = 1/( variance * np.sqrt(2*np.pi) ) * np.exp( -0.5 * ( (x - mean)/variance )**2 )
    
    mean_y = np.arctan(mean)
    dx = np.linspace(mean-3*variance, mean+3*variance, 100)
    m = 1 / ( 1 + mean**2 )
    dy = mean_y + m * (dx-mean)
    
    var_y = m * variance
    y = np.linspace(-3,3, 100)
    pdf2 = 1/( var_y * np.sqrt(2*np.pi) ) * np.exp( -0.5 * ( (y - mean_y)/var_y )**2 )
    
    width = 0.1
    height = 0.2
    left = 0.1
    bottom = 0.1
    
    spacing = 0.005

    rect_scatter = [left+width+spacing, bottom+height+spacing, 0.65, 0.65]
    rect_pdfx = [left+width+spacing, bottom, 0.65, 0.2]
    rect_pdfy = [left, bottom+height+spacing, width, 0.65]

    fig = plt.figure(figsize=(18,8))
    
    ax = fig.add_axes(rect_scatter)
    ax_pdfx = fig.add_axes(rect_pdfx, sharex=ax)
    ax_pdfy = fig.add_axes(rect_pdfy, sharey=ax)
    
    ax.tick_params(axis='x', labelbottom=False)
    ax.tick_params(axis='y', labelleft=False)
    ax_pdfx.tick_params(axis="y", labelleft=False, labelright=True)
    ax_pdfy.tick_params(axis="x", labelbottom=False, labeltop=True)
    
    ax.plot(x, np.arctan(x), '-.')
    ax.plot(dx, dy, 'k')
    ax.set_ylim(-3, 3)
    ax.set_xlim(-10,10)
    ax.vlines(mean, -3, mean_y, colors='k', linestyle='dotted')
    ax.vlines(mean-variance, -3, mean_y-var_y, color='grey', linestyle='-.')
    ax.vlines(mean+variance, -3, mean_y+var_y, color='grey', linestyle='-.')
    ax.hlines(mean_y, -10, mean, colors='k', linestyle='dotted')
    ax.hlines(mean_y-var_y, -10, mean-variance, color='grey', linestyle='-.')
    ax.hlines(mean_y+var_y, -10, mean+variance, color='grey', linestyle='-.')
    t = ax.text(mean-0.2, mean_y+0.2, '$\\frac{df}{dx}=$' + '{0:.2}'.format(m), ha="right", va="bottom", size=15,bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="b", lw=2))
    t = ax.text(9.25, 1, '$f = arctan(x)$', ha="right", va="bottom", size=15,bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="b", lw=2))
    
    ax_pdfx.plot(x, pdf1)
    ax_pdfx.set_ylabel('$PDF_x$')
    ax_pdfx.set_xlim(-10,10)
    ax_pdfx.axvline(mean, color='k', linestyle='dotted')
    ax_pdfx.axvline(mean-variance, color='grey', linestyle='-.')
    ax_pdfx.axvline(mean+variance, color='grey', linestyle='-.')
    
    ax_pdfy.plot(pdf1, dy)
    ax_pdfy.set_xlabel('$PDF_y$')
    ax_pdfy.semilogx()
    ax_pdfy.set_xlim(1e-9,13)
    ax_pdfy.set_ylim(-3, 3)
    ax_pdfy.axhline(mean_y, color='k', linestyle='dotted')
    ax_pdfy.axhline(mean_y-var_y, color='grey', linestyle='-.')
    ax_pdfy.axhline(mean_y+var_y, color='grey', linestyle='-.')
    
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='mean', max=10.0, min=-10.0), FloatSlider(value=1.0, …

# WIP: Including the second derivative
$$
    \frac{d^2f}{dx^2} = \frac{2x}{(1 + x^2)^2} = \frac{2x}{1 + 2x^2 + x^4}
$$
$$
    \mu_y = arctan(\mu_x) \\
    PDF_2(y) = PDF_1(2 \cdot g(-x)) \\
    g(x) = \mu_x + \left.\frac{df}{dx}\right|_{\mu_x} (x - \mu_x) + \frac{1}{2} \left.\frac{d^2f}{dx^2}\right|_{\mu_x} (x - \mu_x)^2
$$

In [3]:
@interact(mean=(-10.0,10.0), variance=(1e-9,6.0))
def show_quad(mean=0.5, variance=1.0):

    x = np.linspace(-10,10, 100)

    pdf1 = 1/( variance * np.sqrt(2*np.pi) ) * np.exp( -0.5 * ( (x - mean)/variance )**2 )
    
    mean_y = np.arctan(mean)
    dx = np.linspace(mean-3*variance, mean+3*variance, 100)
    m = 1 / ( 1 + mean**2 )
    m2 = 2 * mean / ( 1 + mean**2 )**2
    dy = mean_y + m * (dx-mean) + m2/2 * (dx-mean)**2
    
    var_y_left = mean_y + m * (dx[0]-mean) + m2/2 * (dx[0]-mean)**2
    var_y_right =  mean_y + m * (mean+variance) + m2/2 * (mean+variance)**2
    y = np.linspace(-3,3, 100)
    #pdf2 =  1/( variance * np.sqrt(2*np.pi) ) * np.exp( -0.5 * ( (2*dy[::-1] - mean)/variance )**2 )
    
    width = 0.1
    height = 0.2
    left = 0.1
    bottom = 0.1
    
    spacing = 0.005

    rect_scatter = [left+width+spacing, bottom+height+spacing, 0.65, 0.65]
    rect_pdfx = [left+width+spacing, bottom, 0.65, 0.2]
    rect_pdfy = [left, bottom+height+spacing, width, 0.65]

    fig = plt.figure(figsize=(18,8))
    
    ax = fig.add_axes(rect_scatter)
    ax_pdfx = fig.add_axes(rect_pdfx, sharex=ax)
    ax_pdfy = fig.add_axes(rect_pdfy, sharey=ax)
    
    ax.tick_params(axis='x', labelbottom=False)
    ax.tick_params(axis='y', labelleft=False)
    ax_pdfx.tick_params(axis="y", labelleft=False, labelright=True)
    ax_pdfy.tick_params(axis="x", labelbottom=False, labeltop=True)
    
    ax.plot(x, np.arctan(x), '-.')
    ax.plot(dx, dy, 'k')
    ax.set_ylim(-3, 3)
    ax.set_xlim(-10,10)
    
    ax.vlines(mean, -3, mean_y, colors='k', linestyle='dotted')
    ax.vlines(dx[33], -3, dy[33], color='grey', linestyle='-.')
    ax.vlines(dx[-34], -3, dy[-34], color='grey', linestyle='-.')
    
    ax.hlines(mean_y, -10, mean, colors='k', linestyle='dotted')
    ax.hlines(dy[33], -10, dx[33], color='grey', linestyle='-.')
    ax.hlines(dy[-34], -10, dx[-34], color='grey', linestyle='-.')
    
    t = ax.text(mean-0.2, mean_y+0.2, '$\\frac{df}{dx}=$' + '{0:.2}'.format(m) + '\n $\\frac{d^2f}{dx^2}=$' + '{0:.2}'.format(m2), ha="right", va="bottom", size=15,bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="b", lw=2))
    t = ax.text(9.25, 1, '$f = arctan(x)$', ha="right", va="bottom", size=15,bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="b", lw=2))
    
    ax_pdfx.plot(x, pdf1)
    ax_pdfx.set_ylabel('$PDF_x$')
    ax_pdfx.set_xlim(-10,10)
    ax_pdfx.axvline(mean, color='k', linestyle='dotted')
    ax_pdfx.axvline(mean-variance, color='grey', linestyle='-.')
    ax_pdfx.axvline(mean+variance, color='grey', linestyle='-.')
    
    ax_pdfy.plot(pdf1, dy)
    ax_pdfy.set_xlabel('$PDF_y$')
    #ax_pdfy.semilogx()
    #ax_pdfy.set_xlim(1e-9,13)
    ax_pdfy.set_ylim(-3, 3)
    ax_pdfy.axhline(mean_y, color='k', linestyle='dotted')
    ax_pdfy.axhline(dy[33], color='grey', linestyle='-.')
    ax_pdfy.axhline(dy[-34], color='grey', linestyle='-.')
    
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='mean', max=10.0, min=-10.0), FloatSlider(value=1.0, …